In [2]:
import pandas as pd
import os

In [189]:
import os
import pandas as pd
import json

def process_json_to_csv(input_folder, output_csv):
    data = []

    for file_name in os.listdir(input_folder):
        if file_name.endswith(".json"): 
            file_path = os.path.join(input_folder, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:

                json_data = json.load(file)
                
                # Extrair as informações principais
                codigo_universidade = json_data.get("codigo_universidade", "")
                nome_universidade = json_data.get("nome_universidade", "")
                
                # Dividir o nome_universidade para extrair a universidade e a escola
                partes_universidade = nome_universidade.split(" - ")
                universidade = partes_universidade[1] if len(partes_universidade) > 1 else ""
                escola = partes_universidade[2] if len(partes_universidade) > 2 else ""
                
                for curso in json_data.get("cursos", []):
                    codigo_curso = curso.get("codigo_curso", "")
                    
                    # Dividir o nome_curso para extrair apenas o nome do curso
                    partes_curso = curso.get("nome_curso", "").split(" - ")
                    nome_curso = " - ".join(partes_curso[1:]) if len(partes_curso) > 1 else ""
                    
                    # Verificar se o curso é regime pós-laboral
                    if "(regime pós-laboral)" in nome_curso:
                        regime_pos_laboral = 1
                        nome_curso = nome_curso.replace("(regime pós-laboral)", "").strip()
                    else:
                        regime_pos_laboral = 0

                    if "(regime noturno)" in nome_curso:
                        regime_noturno = 1
                        nome_curso = nome_curso.replace("(regime noturno)", "").strip()
                    else:
                        regime_noturno = 0

                    for colocado in curso.get("colocados", []):
                        # Adicionar ao dataset
                        data.append({
                            "tipo_ensino": "politecnico",
                            "codigo_estabelecimento": codigo_universidade,
                            "estabelecimento": universidade.strip(),
                            "escola": escola.strip(),
                            "codigo_curso": codigo_curso,
                            "curso": nome_curso.strip(),
                            "colocado": colocado.get("nome_aluno", ""),
                            "codigo_aluno": colocado.get("codigo_aluno", ""),
                            "regime_pos_laboral": regime_pos_laboral,
                            "regime_noturno": regime_noturno  # 1 ou 0 dependendo do caso
                        })
    
    # Criar DataFrame do Pandas
    df_universidades = pd.DataFrame(data)

    # Salvar em CSV
    df_universidades.to_csv(output_csv, index=False, encoding='utf-8')
    print(f"Arquivo CSV salvo em: {output_csv}")

# Uso da função
process_json_to_csv("dados_politecnicos", "politecnicos.csv")


Arquivo CSV salvo em: politecnicos.csv


In [190]:
df=pd.read_csv('politecnicos.csv')
df.head()

,tipo_ensino,codigo_estabelecimento,estabelecimento,escola,codigo_curso,curso,colocado,codigo_aluno,regime_pos_laboral,regime_noturno
0,politecnico,1320,Universidade da Madeira,Escola Superior de Saúde,9500,Enfermagem,ANDRÉ GUILHERME FERNANDES MENDONÇA,150(...)47,0,0
1,politecnico,1320,Universidade da Madeira,Escola Superior de Saúde,9500,Enfermagem,BEATRIZ MADALENA PEREIRA MACIEIRA,151(...)29,0,0
2,politecnico,1320,Universidade da Madeira,Escola Superior de Saúde,9500,Enfermagem,BEATRIZ MARIA FRANCISCO SOUSA,150(...)18,0,0
3,politecnico,1320,Universidade da Madeira,Escola Superior de Saúde,9500,Enfermagem,BEATRIZ MARIA GOMES FREITAS,156(...)75,0,0
4,politecnico,1320,Universidade da Madeira,Escola Superior de Saúde,9500,Enfermagem,BRUNA RAQUEL FERNANDES DOS SANTOS BELO,149(...)98,0,0


In [4]:
df_filtrado = df[
    (df['curso'].str.contains('medicina', case=False))
]

# Exibindo o DataFrame filtrado
df_filtrado.head()


,tipo_ensino,codigo_estabelecimento,estabelecimento,escola,codigo_curso,curso,colocado,regime_pos_laboral,regime_noturno
17223,politecnico,7040,Instituto Politécnico da Guarda,Escola Superior de Saúde da Guarda,L101,Biotecnologia Medicinal,AFONSO SALVATORI RAMALHO MONIZ AZEVEDO,0,0
17224,politecnico,7040,Instituto Politécnico da Guarda,Escola Superior de Saúde da Guarda,L101,Biotecnologia Medicinal,ANA DE JESUS OLIVEIRA,0,0
17225,politecnico,7040,Instituto Politécnico da Guarda,Escola Superior de Saúde da Guarda,L101,Biotecnologia Medicinal,ANTÓNIO DANIEL TORRÃO GARRIDO TORRES,0,0
17226,politecnico,7040,Instituto Politécnico da Guarda,Escola Superior de Saúde da Guarda,L101,Biotecnologia Medicinal,BEATRIZ CALDEIRA ASSUDE,0,0
17227,politecnico,7040,Instituto Politécnico da Guarda,Escola Superior de Saúde da Guarda,L101,Biotecnologia Medicinal,DAVIDE MIGUEL GANHÃO BARBOSA,0,0


In [134]:

# Filtrando linhas onde 'colocado' ou 'curso' são nulos ou vazios
df_filtrado = df[df['curso'].isna() | (df['colocado'] == '') | df['curso'].isna() | (df['curso'] == '')]
df_filtrado.head()


,tipo_ensino,codigo_estabelecimento,estabelecimento,escola,codigo_curso,curso,colocado,regime_pos_laboral,regime_noturno


In [135]:
# Filtrando o DataFrame por 'codigo_universidade' e 'codigo_curso'
df_filtrado = df[(df['codigo_estabelecimento'] == 3041) & (df['codigo_curso'] == '9085')]


df_filtrado.head()

,tipo_ensino,codigo_estabelecimento,estabelecimento,escola,codigo_curso,curso,colocado,regime_pos_laboral,regime_noturno
1718,universitario,3041,Instituto Politécnico de Bragança,Escola Superior Agrária de Bragança,9085,Enfermagem Veterinária,AFONSO SILVA AFONSO,0,0
1719,universitario,3041,Instituto Politécnico de Bragança,Escola Superior Agrária de Bragança,9085,Enfermagem Veterinária,ANA CAROLINA ALVES PRATAS MANO,0,0
1720,universitario,3041,Instituto Politécnico de Bragança,Escola Superior Agrária de Bragança,9085,Enfermagem Veterinária,ANA ISABEL CORREIA MARTINS,0,0
1721,universitario,3041,Instituto Politécnico de Bragança,Escola Superior Agrária de Bragança,9085,Enfermagem Veterinária,ANA ISABEL OLIVEIRA COSTA,0,0
1722,universitario,3041,Instituto Politécnico de Bragança,Escola Superior Agrária de Bragança,9085,Enfermagem Veterinária,BEATRIZ DA CONCEIÇÃO SOUSA MAGALHÃES,0,0


In [82]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   tipo_ensino             0 non-null      object
 1   codigo_estabelecimento  0 non-null      int64 
 2   estabelecimento         0 non-null      object
 3   escola                  0 non-null      object
 4   codigo_curso            0 non-null      object
 5   curso                   0 non-null      object
 6   colocado                0 non-null      object
 7   regime_pos_laboral      0 non-null      int64 
 8   regime_noturno          0 non-null      int64 
dtypes: int64(3), object(6)
memory usage: 0.0+ bytes


In [3]:
df_uni=pd.read_csv('universidades.csv')
df_poli=pd.read_csv('politecnicos.csv')

In [192]:
df_uni.head(1)

,tipo_ensino,codigo_estabelecimento,estabelecimento,escola,codigo_curso,curso,colocado,codigo_aluno,regime_pos_laboral,regime_noturno
0,universitario,140,Universidade dos Açores,Faculdade de Ciências Agrárias e do Ambiente,9022,Ciências Agrárias,BERNARDO COUTO MATOS,150(...)35,0,0


In [106]:
df_poli.head(1)

,tipo_ensino,codigo_estabelecimento,estabelecimento,escola,codigo_curso,curso,colocado,regime_pos_laboral,regime_noturno
0,politecnico,1320,Universidade da Madeira,Escola Superior de Saúde,9500,Enfermagem,ANDRÉ GUILHERME FERNANDES MENDONÇA,0,0


In [101]:
df_poli.shape, df_uni.shape

((19789, 9), (30174, 9))

In [6]:
df_concatenado = pd.concat([df_uni, df_poli], ignore_index=True)

In [103]:
df_concatenado.shape

(49963, 9)

In [ ]:
df_concatenado.to_csv('table_colocados.csv', index=False, encoding='utf-8')

print("Arquivo CSV salvo com sucesso!")

Arquivo CSV salvo com sucesso!


# Criptografia

In [89]:
df=pd.read_csv('table_colocados.csv')

In [109]:
import hashlib
import os
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend

def hash_file(filepath, algorithm="sha256"):
    """Calcula a hash de um único arquivo"""
    hash_func = hashlib.new(algorithm)
    with open(filepath, "rb") as f:
        while chunk := f.read(8192):  # Lê em pedaços de 8KB
            hash_func.update(chunk)
    return hash_func.hexdigest()

def generate_keys():
    """Gera as chaves pública e privada"""
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()
    return private_key, public_key

def sign_hash(private_key, hash_value):
    """Assina a hash com a chave privada"""
    signature = private_key.sign(
        hash_value.encode(),
        padding.PKCS1v15(),
        hashes.SHA256()
    )
    return signature

def save_public_key(public_key, filename):
    """Salva a chave pública em formato PEM"""
    pem = public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    with open(filename, "wb") as f:
        f.write(pem)

def save_signature(signature, filename):
    """Salva a assinatura digital em um arquivo binário"""
    with open(filename, "wb") as f:
        f.write(signature)


# 1. Calcular a hash da tabela
tabela_hash = hash_file("table_colocados.csv")
print("Hash da tabela:", tabela_hash)

# 2. Gerar as chaves pública e privada
private_key, public_key = generate_keys()

# 3. Assinar a hash da tabela com a chave privada
signature = sign_hash(private_key, tabela_hash)
print("Assinatura digital:", signature.hex())

save_public_key(public_key, "public_key_tabela.pem")
print("Chave pública salva em 'public_key_tabela.pem'")

# 5. Salvar a assinatura digital em um arquivo
save_signature(signature, "signature_tabela.bin")
print("Assinatura salva em 'signature_tabela.bin'")

Hash da tabela: 4e4e2dafb62c7da65605588d21a8a4c61911102547ff50b0c8ad9f5797c254fc
Assinatura digital: af1addca8b3d3c07c53c48d50a30d30e08828240c5aff8ae65546e55746dc1907aa0e60186c8a6d853d2bdf691748568a9f4973005ab4ad2ec8114d953e33e4b65552481539824de7b5c505e06eed4911a3e3fb1ed778cf19f1fa1b516117d4038c8a2babba310183aa6b4eed34d662867f77f01afd9d5ad743b76278124dc1e240ac171e6a15d60c4a4d768f7b3a8b2afbc2f9b09baa1f18ffd8521ba738a29e74aa1264d710ddd3d4ff5cb8f932c7c848283cd6d1b7ae97f9baddaa124a5521eb59aa435a145e92b4f2334dd47fcb0f13fa07fd8f4c2277ae4a461ee366cd22cda85f6a669a1f82c86f8389ff08050388131ed67b6d05657a00fa4efd91f9f
Chave pública salva em 'public_key_tabela.pem'
Assinatura salva em 'signature_tabela.bin'


## Elisa

In [4]:
import pandas as pd

In [5]:
candidatos=pd.read_csv('candidatos_elisa.csv')

In [9]:
candidatos.head()

,Código Estabelecimento,Estabelecimento,Código Curso,Curso,Nº Ordem (parcial),Nº Ident.,Nome,Nota,Opção,PI,12º,10º/11º
0,602,Universidade de Évora - Escola de Ciências e T...,9003,Agronomia,1,151(...)15,MARTA DO VALE MARTINS,"188,7",6,"177,0",195,195
1,602,Universidade de Évora - Escola de Ciências e T...,9003,Agronomia,2,156(...)35,FRANCISCO ISIDORO AROEIRA,"177,8",1,"181,0",176,176
2,602,Universidade de Évora - Escola de Ciências e T...,9003,Agronomia,3,308(...)62,ANDRÉ FILIPE SILVA RODRIGUES,"177,0",4,"188,0",171,171
3,602,Universidade de Évora - Escola de Ciências e T...,9003,Agronomia,4,154(...)43,MARIA CONSTANÇA GUERREIRO MORENO,"171,9",4,"168,0",174,174
4,602,Universidade de Évora - Escola de Ciências e T...,9003,Agronomia,5,306(...)79,AFONSO PRATAS ROSA,"169,0",2,"156,0",176,176


In [92]:
def separar_universidade_escola(estabelecimento):
    partes = estabelecimento.split(' - ')
    universidade = partes[0] if len(partes) > 0 else ""
    escola = partes[1] if len(partes) > 1 else ""
    return universidade, escola

# Aplicando a função para dividir as colunas
candidatos[['estabelecimento', 'escola']] = candidatos['Estabelecimento'].apply(
    lambda x: pd.Series(separar_universidade_escola(x))
)

In [93]:
candidatos = candidatos.drop(['Estabelecimento'], axis=1)

In [28]:
df_filtrado = candidatos[
    (candidatos['Curso'].str.contains('regime', case=False))
]

# Exibindo o DataFrame filtrado
df_filtrado.head()

,Código Estabelecimento,Código Curso,Curso,Nº Ordem (parcial),Nº Ident.,Nome,Nota,Opção,PI,12º,10º/11º,estabelecimento,escola
1682,902,8109,Sociologia (regime pós-laboral),1,152(...)97,JOANA ANA LAMURIAS FERREIRA,"189,0",6,"186,0",192,192,Universidade Nova de Lisboa,Faculdade de Ciências Sociais e Humanas
1683,902,8109,Sociologia (regime pós-laboral),2,155(...)13,ANA YOLENY CARDOSO CALENGA,"183,5",2,"192,0",175,175,Universidade Nova de Lisboa,Faculdade de Ciências Sociais e Humanas
1684,902,8109,Sociologia (regime pós-laboral),3,309(...)35,DENISE LUANA GONÇALVES AMADO,"171,5",3,"187,0",156,156,Universidade Nova de Lisboa,Faculdade de Ciências Sociais e Humanas
1685,902,8109,Sociologia (regime pós-laboral),4,159(...)30,BEATRIZ VALENÇA DUARTE CRUZ PEREIRA,"161,5",3,"159,0",164,164,Universidade Nova de Lisboa,Faculdade de Ciências Sociais e Humanas
1686,902,8109,Sociologia (regime pós-laboral),5,240(...)06,GISELE RIBEIRO DA SILVA SANTOS,"161,0",5,"157,0",165,165,Universidade Nova de Lisboa,Faculdade de Ciências Sociais e Humanas


In [94]:
def processar_regime(curso):
    regime_pos_laboral = 1 if '(regime pós-laboral)' in curso else 0
    regime_noturno = 1 if '(regime noturno)' in curso else 0
    curso = curso.replace('(regime pós-laboral)', '').replace('(regime noturno)', '').strip()
    return curso, regime_pos_laboral, regime_noturno

# Aplicando a função para gerar as novas colunas
candidatos[['curso', 'regime_pos_laboral', 'regime_noturno']] = candidatos['Curso'].apply(
    lambda x: pd.Series(processar_regime(x))
)

In [200]:
candidatos.head()

,Código Estabelecimento,Código Curso,Curso,Nº Ordem (parcial),Nº Ident.,Nome,Nota,Opção,PI,12º,10º/11º,estabelecimento,escola,curso,regime_pos_laboral,regime_noturno
0,602,9003,Agronomia,1,151(...)15,MARTA DO VALE MARTINS,"188,7",6,"177,0",195,195,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0
1,602,9003,Agronomia,2,156(...)35,FRANCISCO ISIDORO AROEIRA,"177,8",1,"181,0",176,176,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0
2,602,9003,Agronomia,3,308(...)62,ANDRÉ FILIPE SILVA RODRIGUES,"177,0",4,"188,0",171,171,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0
3,602,9003,Agronomia,4,154(...)43,MARIA CONSTANÇA GUERREIRO MORENO,"171,9",4,"168,0",174,174,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0
4,602,9003,Agronomia,5,306(...)79,AFONSO PRATAS ROSA,"169,0",2,"156,0",176,176,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0


In [95]:
candidatos = candidatos.drop(['Curso'], axis=1)

In [96]:
candidatos.rename(columns={
    'Código Estabelecimento': 'codigo_estabelecimento',
    'Código Curso': 'codigo_curso',
    'Nº Ident.': 'cc'
}, inplace=True)


In [203]:
candidatos.head()

,codigo_estabelecimento,codigo_curso,Nº Ordem (parcial),cc,Nome,Nota,Opção,PI,12º,10º/11º,estabelecimento,escola,curso,regime_pos_laboral,regime_noturno
0,602,9003,1,151(...)15,MARTA DO VALE MARTINS,"188,7",6,"177,0",195,195,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0
1,602,9003,2,156(...)35,FRANCISCO ISIDORO AROEIRA,"177,8",1,"181,0",176,176,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0
2,602,9003,3,308(...)62,ANDRÉ FILIPE SILVA RODRIGUES,"177,0",4,"188,0",171,171,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0
3,602,9003,4,154(...)43,MARIA CONSTANÇA GUERREIRO MORENO,"171,9",4,"168,0",174,174,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0
4,602,9003,5,306(...)79,AFONSO PRATAS ROSA,"169,0",2,"156,0",176,176,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0


In [97]:
candidatos['candidato'] = 1

In [98]:
candidatos.head()

,codigo_estabelecimento,codigo_curso,Nº Ordem (parcial),cc,Nome,Nota,Opção,PI,12º,10º/11º,estabelecimento,escola,curso,regime_pos_laboral,regime_noturno,candidato
0,602,9003,1,151(...)15,MARTA DO VALE MARTINS,"188,7",6,"177,0",195,195,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1
1,602,9003,2,156(...)35,FRANCISCO ISIDORO AROEIRA,"177,8",1,"181,0",176,176,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1
2,602,9003,3,308(...)62,ANDRÉ FILIPE SILVA RODRIGUES,"177,0",4,"188,0",171,171,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1
3,602,9003,4,154(...)43,MARIA CONSTANÇA GUERREIRO MORENO,"171,9",4,"168,0",174,174,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1
4,602,9003,5,306(...)79,AFONSO PRATAS ROSA,"169,0",2,"156,0",176,176,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1


## Mesclar

In [7]:
colocados=pd.read_csv('table_colocados.csv')

In [208]:
colocados.head()

,tipo_ensino,codigo_estabelecimento,estabelecimento,escola,codigo_curso,curso,colocado,codigo_aluno,regime_pos_laboral,regime_noturno
0,universitario,140,Universidade dos Açores,Faculdade de Ciências Agrárias e do Ambiente,9022,Ciências Agrárias,BERNARDO COUTO MATOS,150(...)35,0,0
1,universitario,140,Universidade dos Açores,Faculdade de Ciências Agrárias e do Ambiente,9022,Ciências Agrárias,ISAC SILVA MENDONÇA,158(...)81,0,0
2,universitario,140,Universidade dos Açores,Faculdade de Ciências Agrárias e do Ambiente,9022,Ciências Agrárias,MÁRCIA DUARTE CARDOSO,307(...)38,0,0
3,universitario,140,Universidade dos Açores,Faculdade de Ciências Agrárias e do Ambiente,9022,Ciências Agrárias,MARIA MENDES GREGÓRIO,150(...)99,0,0
4,universitario,140,Universidade dos Açores,Faculdade de Ciências Agrárias e do Ambiente,9022,Ciências Agrárias,MIGUEL ÂNGELO VIVEIROS MARTINS,301(...)83,0,0


In [8]:
colocados = colocados.drop(['tipo_ensino'], axis=1)

In [9]:
colocados.rename(columns={
    'colocado': 'Nome',
    'codigo_aluno': 'cc'
}, inplace=True)

In [10]:
colocados['candidato'] = 0

In [23]:
colocados.head()

,codigo_estabelecimento,estabelecimento,escola,codigo_curso,curso,Nome,cc,regime_pos_laboral,regime_noturno,candidato
0,140,Universidade dos Açores,Faculdade de Ciências Agrárias e do Ambiente,9022,Ciências Agrárias,BERNARDO COUTO MATOS,150(...)35,0,0,0
1,140,Universidade dos Açores,Faculdade de Ciências Agrárias e do Ambiente,9022,Ciências Agrárias,ISAC SILVA MENDONÇA,158(...)81,0,0,0
2,140,Universidade dos Açores,Faculdade de Ciências Agrárias e do Ambiente,9022,Ciências Agrárias,MÁRCIA DUARTE CARDOSO,307(...)38,0,0,0
3,140,Universidade dos Açores,Faculdade de Ciências Agrárias e do Ambiente,9022,Ciências Agrárias,MARIA MENDES GREGÓRIO,150(...)99,0,0,0
4,140,Universidade dos Açores,Faculdade de Ciências Agrárias e do Ambiente,9022,Ciências Agrárias,MIGUEL ÂNGELO VIVEIROS MARTINS,301(...)83,0,0,0


In [211]:
colocados.shape, candidatos.shape

((49963, 11), (280965, 16))

In [11]:
df_concat = pd.concat([candidatos, colocados], ignore_index=True)

In [36]:
df_concat.shape

(330928, 15)

In [45]:
df_concat.head()

,codigo_estabelecimento,codigo_curso,Nº Ordem (parcial),Nome,Nota,Opção,PI,12º,10º/11º,estabelecimento,escola,curso,regime_pos_laboral,regime_noturno,candidato
0,602,9003,1.0,MARTA DO VALE MARTINS,188.7,6.0,177.0,195.0,195.0,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1
1,602,9003,2.0,FRANCISCO ISIDORO AROEIRA,177.8,1.0,181.0,176.0,176.0,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1
2,602,9003,3.0,ANDRÉ FILIPE SILVA RODRIGUES,177.0,4.0,188.0,171.0,171.0,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1
3,602,9003,4.0,MARIA CONSTANÇA GUERREIRO MORENO,171.9,4.0,168.0,174.0,174.0,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1
4,602,9003,5.0,AFONSO PRATAS ROSA,169.0,2.0,156.0,176.0,176.0,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1


In [219]:
df_concat['Nota'] = df_concat['Nota'].str.replace(',', '.').astype(float)
df_concat['PI'] = df_concat['PI'].str.replace(',', '.').astype(float)

AttributeError: Can only use .str accessor with string values!

In [12]:
candidatos['Nota'] = candidatos['Nota'].str.replace(',', '.').astype(float)
candidatos['PI'] = candidatos['PI'].str.replace(',', '.').astype(float)

In [214]:
df_com_letras = df_concat[df_concat['codigo_curso'].str.contains('[a-zA-Z]', na=False)]
df_com_letras.head()

,codigo_estabelecimento,codigo_curso,Nº Ordem (parcial),cc,Nome,Nota,Opção,PI,12º,10º/11º,estabelecimento,escola,curso,regime_pos_laboral,regime_noturno,candidato,tipo_ensino
1561,602,L090,1.0,158(...)35,FRANCISCO SOBRAL RAMALHO,167.3,5.0,166.0,168.0,168.0,Universidade de Évora,Escola de Ciências e Tecnologia,Ecologia e Ambiente,0,0,1,NaN
1562,602,L090,2.0,309(...)81,MARGARIDA CORREIA VIEIRA,155.8,6.0,135.0,167.0,167.0,Universidade de Évora,Escola de Ciências e Tecnologia,Ecologia e Ambiente,0,0,1,NaN
1563,602,L090,3.0,152(...)44,RUBEN MANUEL PEREIRA GALINHA,148.0,3.0,122.0,162.0,162.0,Universidade de Évora,Escola de Ciências e Tecnologia,Ecologia e Ambiente,0,0,1,NaN
1564,602,L090,4.0,153(...)91,LEONARDO JOSÉ DE ABREU GUERREIRO FARRAJOTA,146.5,3.0,125.0,158.0,158.0,Universidade de Évora,Escola de Ciências e Tecnologia,Ecologia e Ambiente,0,0,1,NaN
1565,602,L090,5.0,306(...)56,MARTA MARIA DE OLIVEIRA CARNEIRO BALTAZAR,146.3,6.0,132.0,154.0,154.0,Universidade de Évora,Escola de Ciências e Tecnologia,Ecologia e Ambiente,0,0,1,NaN


In [220]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330928 entries, 0 to 330927
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   codigo_estabelecimento  330928 non-null  int64  
 1   codigo_curso            330928 non-null  object 
 2   Nº Ordem (parcial)      280965 non-null  float64
 3   cc                      330928 non-null  object 
 4   Nome                    330928 non-null  object 
 5   Nota                    280965 non-null  float64
 6   Opção                   280965 non-null  float64
 7   PI                      280965 non-null  float64
 8   12º                     280965 non-null  float64
 9   10º/11º                 280965 non-null  float64
 10  estabelecimento         330928 non-null  object 
 11  escola                  330928 non-null  object 
 12  curso                   330928 non-null  object 
 13  regime_pos_laboral      330928 non-null  int64  
 14  regime_noturno      

In [13]:
df_concat[df_concat.select_dtypes(include=['object']).columns] = df_concat.select_dtypes(include=['object']).fillna('D')
df_concat[df_concat.select_dtypes(include=['object']).columns] = df_concat[df_concat.select_dtypes(include=['object']).columns].replace('', 'D')

In [222]:
df_concat.to_csv('Tudo_Junto.csv', index=False, encoding='utf-8')

print("Arquivo CSV salvo com sucesso!")

Arquivo CSV salvo com sucesso!


In [223]:
candidatos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280965 entries, 0 to 280964
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   codigo_estabelecimento  280965 non-null  int64 
 1   codigo_curso            280965 non-null  object
 2   Nº Ordem (parcial)      280965 non-null  int64 
 3   cc                      280965 non-null  object
 4   Nome                    280965 non-null  object
 5   Nota                    280965 non-null  object
 6   Opção                   280965 non-null  int64 
 7   PI                      280965 non-null  object
 8   12º                     280965 non-null  int64 
 9   10º/11º                 280965 non-null  int64 
 10  estabelecimento         280965 non-null  object
 11  escola                  280965 non-null  object
 12  curso                   280965 non-null  object
 13  regime_pos_laboral      280965 non-null  int64 
 14  regime_noturno          280965 non-n

In [14]:
candidatos['Nota'] = candidatos['Nota'].str.replace(',', '.').astype(float)
candidatos['PI'] = candidatos['PI'].str.replace(',', '.').astype(float)

AttributeError: Can only use .str accessor with string values!

In [225]:
candidatos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280965 entries, 0 to 280964
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   codigo_estabelecimento  280965 non-null  int64  
 1   codigo_curso            280965 non-null  object 
 2   Nº Ordem (parcial)      280965 non-null  int64  
 3   cc                      280965 non-null  object 
 4   Nome                    280965 non-null  object 
 5   Nota                    280965 non-null  float64
 6   Opção                   280965 non-null  int64  
 7   PI                      280965 non-null  float64
 8   12º                     280965 non-null  int64  
 9   10º/11º                 280965 non-null  int64  
 10  estabelecimento         280965 non-null  object 
 11  escola                  280965 non-null  object 
 12  curso                   280965 non-null  object 
 13  regime_pos_laboral      280965 non-null  int64  
 14  regime_noturno      

In [15]:
candidatos[candidatos.select_dtypes(include=['object']).columns] = candidatos.select_dtypes(include=['object']).fillna('D')
colocados[colocados.select_dtypes(include=['object']).columns] = colocados.select_dtypes(include=['object']).fillna('D')
candidatos[candidatos.select_dtypes(include=['object']).columns] = candidatos[candidatos.select_dtypes(include=['object']).columns].replace('', 'D')

In [105]:
candidatos.to_csv('candidatos.csv')
colocados.to_csv('colocados.csv')

In [16]:
candidatos=pd.read_csv('candidatos.csv')
colocados=pd.read_csv('colocados.csv')

In [17]:
df_merged = pd.merge(candidatos, colocados, 
                     on=['codigo_estabelecimento','codigo_curso', 'curso','Nome', 'estabelecimento' , 'escola', 'regime_pos_laboral', 'regime_noturno', 'cc'], 
                     how='inner')

In [228]:
df_merged.head()

,codigo_estabelecimento,codigo_curso,Nº Ordem (parcial),cc,Nome,Nota,Opção,PI,12º,10º/11º,estabelecimento,escola,curso,regime_pos_laboral,regime_noturno,candidato_x,tipo_ensino,candidato_y
0,602,9003,2,156(...)35,FRANCISCO ISIDORO AROEIRA,177.8,1,181.0,176,176,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1,universitario,0
1,602,9003,11,307(...)84,JORGE DINIS DE JESUS CHARRITO GARCIA,154.9,2,138.0,164,164,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1,universitario,0
2,602,9003,17,159(...)10,FRANCISCA RAMOS COMPRIDO RODRIGUES RATO,150.8,1,115.0,170,170,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1,universitario,0
3,602,9003,19,153(...)42,MADALENA DUARTE VENTURA DA SILVA DIAS,150.1,2,126.0,163,163,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1,universitario,0
4,602,9003,32,152(...)04,CAROLINA LEONOR CARVALHO BOTA OLIVEIRA,140.6,1,136.0,143,143,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1,universitario,0


In [230]:
df_merged.shape, colocados.shape

((49963, 18), (49963, 11))

In [231]:
df_merged.to_csv('candidatos_que_foram_colocados.csv', index=False, encoding='utf-8')

print("Arquivo CSV salvo com sucesso!")

Arquivo CSV salvo com sucesso!


In [53]:
import pandas as pd
df=pd.read_csv('Tudo_Junto.csv')

In [54]:
df.head(2)

,codigo_estabelecimento,codigo_curso,Nº Ordem (parcial),cc,Nome,Nota,Opção,PI,12º,10º/11º,estabelecimento,escola,curso,regime_pos_laboral,regime_noturno,candidato,tipo_ensino
0,602,9003,1.0,151(...)15,MARTA DO VALE MARTINS,188.7,6.0,177.0,195.0,195.0,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1,D
1,602,9003,2.0,156(...)35,FRANCISCO ISIDORO AROEIRA,177.8,1.0,181.0,176.0,176.0,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1,D


In [16]:
valores_unicos = df['estabelecimento'].unique()

# Converter para lista, se necessário
valores_unicos_lista = valores_unicos.tolist()
valores_unicos_lista

['Universidade de Évora',
 'Universidade Nova de Lisboa',
 'Universidade do Porto',
 'Universidade de Trás-os-Montes e Alto Douro',
 'Universidade da Madeira',
 'Universidade de Lisboa',
 'Universidade de Aveiro',
 'Instituto Politécnico do Cávado e do Ave',
 'Instituto Politécnico de Castelo Branco',
 'Universidade do Algarve',
 'Instituto Politécnico de Lisboa',
 'Instituto Politécnico do Porto',
 'Instituto Politécnico de Santarém',
 'Instituto Politécnico de Viana do Castelo',
 'Instituto Politécnico de Setúbal',
 'Instituto Politécnico de Tomar',
 'Escola Superior de Hotelaria e Turismo do Estoril',
 'Instituto Politécnico de Viseu',
 'Universidade dos Açores',
 'Escola Superior de Enfermagem de Coimbra',
 'Universidade de Coimbra',
 'Instituto Politécnico de Bragança',
 'Instituto Politécnico de Leiria',
 'ISCTE',
 'Instituto Politécnico de Coimbra',
 'Escola Superior de Enfermagem do Porto',
 'Universidade da Beira Interior',
 'Instituto Politécnico de Beja',
 'Universidade de C

In [17]:
import requests
import pandas as pd

def get_university_coordinates(api_key, university_name):
    # Define o endpoint da API
    url = "https://api.opencagedata.com/geocode/v1/json"
    
    # Parâmetros para a consulta
    params = {
        'q': university_name + ", Portugal",  # Nome da universidade e o país
        'key': api_key,  # Chave da API
        'language': 'pt',  # Idioma da resposta
        'limit': 1        # Retorna apenas o melhor resultado
    }
    
    try:
        # Fazendo a requisição para a API
        response = requests.get(url, params=params)
        response.raise_for_status()  # Lança um erro se o status HTTP não for 200
        
        # Processa a resposta
        data = response.json()
        if data['results']:
            # Obtendo as coordenadas do primeiro resultado
            coordinates = data['results'][0]['geometry']
            return {
                'latitude': coordinates['lat'],
                'longitude': coordinates['lng']
            }
        else:
            return {'latitude': None, 'longitude': None}
    
    except requests.exceptions.RequestException as e:
        return {'latitude': None, 'longitude': None, 'erro': str(e)}

# Função para criar o CSV com as coordenadas
def create_university_coordinates_csv(api_key, valores_unicos_lista):
    # Lista para armazenar os dados para o CSV
    dados = []
    
    for universidade in valores_unicos_lista:
        print(f"Obtendo coordenadas para: {universidade}")
        coordenadas = get_university_coordinates(api_key, universidade)
        dados.append({
            'estabelecimento': universidade,
            'latitude': coordenadas['latitude'],
            'longitude': coordenadas['longitude']
        })
    
    # Criando o DataFrame e salvando como CSV
    df = pd.DataFrame(dados)
    df.to_csv('universidades_com_coordenadas.csv', index=False)
    print("CSV criado com sucesso: universidades_com_coordenadas.csv")

# Exemplo de uso
if __name__ == "__main__":
    api_key = "cae41cd704434a14b10f6d03c001fae1"  # Insira sua chave da API do OpenCage
    # Supondo que 'valores_unicos_lista' seja a lista de universidades
    valores_unicos_lista = valores_unicos_lista
    
    create_university_coordinates_csv(api_key, valores_unicos_lista)



Obtendo coordenadas para: Universidade de Évora
Obtendo coordenadas para: Universidade Nova de Lisboa
Obtendo coordenadas para: Universidade do Porto
Obtendo coordenadas para: Universidade de Trás-os-Montes e Alto Douro
Obtendo coordenadas para: Universidade da Madeira
Obtendo coordenadas para: Universidade de Lisboa
Obtendo coordenadas para: Universidade de Aveiro
Obtendo coordenadas para: Instituto Politécnico do Cávado e do Ave
Obtendo coordenadas para: Instituto Politécnico de Castelo Branco
Obtendo coordenadas para: Universidade do Algarve
Obtendo coordenadas para: Instituto Politécnico de Lisboa
Obtendo coordenadas para: Instituto Politécnico do Porto
Obtendo coordenadas para: Instituto Politécnico de Santarém
Obtendo coordenadas para: Instituto Politécnico de Viana do Castelo
Obtendo coordenadas para: Instituto Politécnico de Setúbal
Obtendo coordenadas para: Instituto Politécnico de Tomar
Obtendo coordenadas para: Escola Superior de Hotelaria e Turismo do Estoril
Obtendo coorde

In [34]:
local=pd.read_csv('universidades_com_coordenadas.csv')



In [41]:
candidatos_por_est = candidatos.groupby('estabelecimento')['candidato'].count().reset_index()

# Renomear a coluna 'candidato' para 'total_candidatos'
candidatos_por_est = candidatos_por_est.rename(columns={'candidato': 'total_candidatos'})
candidatos_por_est

colocados_por_est = colocados.groupby('estabelecimento')['candidato'].count().reset_index()

# Renomear a coluna 'candidato' para 'total_candidatos'
colocados_por_est = colocados_por_est.rename(columns={'candidato': 'total_colocados'})
colocados_por_est

,estabelecimento,total_colocados
0,Escola Superior Náutica Infante D. Henrique,157
1,Escola Superior de Enfermagem de Coimbra,311
2,Escola Superior de Enfermagem de Lisboa,279
3,Escola Superior de Enfermagem do Porto,258
4,Escola Superior de Hotelaria e Turismo do Estoril,393
5,ISCTE,1524
6,Instituto Politécnico da Guarda,522
7,Instituto Politécnico de Beja,337
8,Instituto Politécnico de Bragança,999
9,Instituto Politécnico de Castelo Branco,678


In [43]:
result = pd.merge(candidatos_por_est, colocados_por_est, on='estabelecimento', how='left')
result = pd.merge(result, local, on='estabelecimento', how='left')

In [45]:
# Exportando para CSV
result.to_csv('geografia.csv', index=False)


In [1]:
import pandas as pd
cand_col=pd.read_csv('candidatos_que_foram_colocados.csv')

In [2]:
cand_col.head()

,codigo_estabelecimento,codigo_curso,Nº Ordem (parcial),cc,Nome,Nota,Opção,PI,12º,10º/11º,estabelecimento,escola,curso,regime_pos_laboral,regime_noturno,candidato_x,tipo_ensino,candidato_y
0,602,9003,2,156(...)35,FRANCISCO ISIDORO AROEIRA,177.8,1,181.0,176,176,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1,universitario,0
1,602,9003,11,307(...)84,JORGE DINIS DE JESUS CHARRITO GARCIA,154.9,2,138.0,164,164,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1,universitario,0
2,602,9003,17,159(...)10,FRANCISCA RAMOS COMPRIDO RODRIGUES RATO,150.8,1,115.0,170,170,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1,universitario,0
3,602,9003,19,153(...)42,MADALENA DUARTE VENTURA DA SILVA DIAS,150.1,2,126.0,163,163,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1,universitario,0
4,602,9003,32,152(...)04,CAROLINA LEONOR CARVALHO BOTA OLIVEIRA,140.6,1,136.0,143,143,Universidade de Évora,Escola de Ciências e Tecnologia,Agronomia,0,0,1,universitario,0


In [79]:
colocados_sem_1_opcao = cand_col[cand_col['Opção'] != 1]
colunas_desejadas = ['Opção', 'Nome', 'cc', 'curso']
df_selecionado = colocados_sem_1_opcao[colunas_desejadas]
df_selecionado

,Opção,Nome,cc,curso
1,2,JORGE DINIS DE JESUS CHARRITO GARCIA,307(...)84,Agronomia
3,2,MADALENA DUARTE VENTURA DA SILVA DIAS,153(...)42,Agronomia
6,2,JOÃO ANTÓNIO GOMES CAZALTA,303(...)27,Agronomia
7,6,JOSÉ LUÍS BRITO MATADO,307(...)51,Agronomia
11,2,LUCAS SANTIAGO VERA-CRUZ PINTO,301(...)50,Agronomia
...,...,...,...,...
49948,3,LARA FILIPA ALVES FERREIRA,301(...)86,Enfermagem
49949,2,VITÓRIA FRANCISCA FERNANDES DA COSTA,304(...)62,Enfermagem
49951,2,JOSÉ EDUARDO FIGUEIREDO GUEDES,306(...)68,Enfermagem
49952,2,RODRIGO RIBEIRO BARROS,150(...)93,Enfermagem


In [68]:
df=pd.read_csv('Tudo_junto.csv')

In [ ]:
candidatos_1_opcao = candidatos[(df['Opção'] == 1) ]
candidatos_1_opcao
# Selecionando as colunas desejadas
colunas_desejadas = ['Opção', 'Nome', 'cc', 'curso']
# df_selecionado = candidatos_1_opcao[colunas_desejadas]
# df_selecionado

C:\Users\Utilizador\AppData\Local\Temp\ipykernel_10936\4198211148.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  candidatos_1_opcao = candidatos[(df['Opção'] == '1.0') ]
